In [2]:
pip install fitz pymupdf

In [2]:
pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 25.0 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()

Saving Indian Rivers.pdf to Indian Rivers.pdf


In [40]:
from google.colab import files
uploaded = files.upload()

Saving Week 5.pdf to Week 5.pdf


In [4]:
!pip install transformers
!pip install torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Loading SQUAD Dataset


In [5]:
from google.colab import files
uploaded = files.upload()

Saving train-v2.0.json to train-v2.0.json


In [8]:
from google.colab import files
uploaded = files.upload()

Saving dev-v2.0.json to dev-v2.0.json


In [12]:
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.optim import AdamW
from transformers import BertTokenizerFast, BertForQuestionAnswering, get_linear_schedule_with_warmup
import json
import time
import datetime
import numpy as np
import re
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from torch.cuda.amp import autocast, GradScaler

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# Function to extract text from PDF
def extract_text(path):
    import fitz
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [14]:
# Preprocess text by removing URLs, lowercasing, removing stopwords, and tokenizing
def text_preprocessing(text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = text.lower()  # Lowercase text
    tokens = nltk.word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords and non-alphanumeric
    return tokens

In [15]:
# Load SQuAD dataset
def load_squad(path):
    with open(path, 'r') as f:
        squad_data = json.load(f)
    return squad_data

In [16]:
# Preprocess SQuAD data
def preprocess_squad(squad_data):
    contexts = []
    questions = []
    answers = []
    for group in squad_data['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if len(qa['answers']) > 0:
                    answer = qa['answers'][0]['text']  # Take first answer as ground truth
                    start_idx = context.find(answer)
                    end_idx = start_idx + len(answer)
                    contexts.append(context)
                    questions.append(question)
                    answers.append((start_idx, end_idx, answer))
    return contexts, questions, answers

In [17]:
# Create a custom dataset class
class SquadDataset(Dataset):
    def __init__(self, contexts, questions, answers, tokenizer, max_len):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        context = self.contexts[idx]
        question = self.questions[idx]
        start_idx, end_idx, answer = self.answers[idx]
        encoding = self.tokenizer(
            question,
            context,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        # Calculate token start and end positions
        context_encoding = self.tokenizer(context, truncation=True, padding='max_length', max_length=self.max_len, return_offsets_mapping=True)
        offsets = context_encoding['offset_mapping']

        start_positions = []
        end_positions = []

        for idx, (start, end) in enumerate(offsets):
            if start <= start_idx < end:
                start_positions.append(idx)
            if start < end_idx <= end:
                end_positions.append(idx)

        if not start_positions:
            start_positions = [self.max_len]
        if not end_positions:
            end_positions = [self.max_len]

        token_start_idx = start_positions[0]
        token_end_idx = end_positions[-1]

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(token_start_idx, dtype=torch.long),
            'end_positions': torch.tensor(token_end_idx, dtype=torch.long)
        }

In [18]:
# Train function
def train_model(model, data_loader, optimizer, scheduler, scaler, device, epoch):
    model = model.train()
    losses = []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        with autocast():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions
            )
            loss = outputs.loss
            losses.append(loss.item())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad()

    return np.mean(losses)

In [19]:
# Evaluation function
def eval_model(model, data_loader, device):
    model = model.eval()
    losses = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions
            )
            loss = outputs.loss
            losses.append(loss.item())

    return np.mean(losses)

In [20]:
# Main function to train and evaluate the model
def main():
    # Set up parameters
    MAX_LEN = 384
    BATCH_SIZE = 8
    EPOCHS = 2
    LEARNING_RATE = 2e-5
    WARMUP_STEPS = 0
    WEIGHT_DECAY = 0.01

    # Load SQuAD dataset
    squad_train_path = 'train-v2.0.json'
    squad_val_path = 'dev-v2.0.json'
    squad_train_data = load_squad(squad_train_path)
    squad_val_data = load_squad(squad_val_path)
    train_contexts, train_questions, train_answers = preprocess_squad(squad_train_data)
    val_contexts, val_questions, val_answers = preprocess_squad(squad_val_data)

    # Load BERT tokenizer and model
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    model = BertForQuestionAnswering.from_pretrained('bert-base-uncased', return_dict=True)

    # Create datasets and dataloaders
    train_dataset = SquadDataset(train_contexts, train_questions, train_answers, tokenizer, MAX_LEN)
    val_dataset = SquadDataset(val_contexts, val_questions, val_answers, tokenizer, MAX_LEN)
    train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=RandomSampler(train_dataset))
    val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=SequentialSampler(val_dataset))

    # Set up optimizer, scheduler, and mixed precision scaler
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    total_steps = len(train_data_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_steps)
    scaler = GradScaler()

    # Use GPU if available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)

    # Training loop
    for epoch in range(EPOCHS):
        start_time = time.time()
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, scaler, device, epoch)
        val_loss = eval_model(model, val_data_loader, device)
        end_time = time.time()

        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print(f'Train loss: {train_loss}')
        print(f'Validation loss: {val_loss}')
        print(f'Epoch duration: {datetime.timedelta(seconds=end_time - start_time)}')

    # Save the fine-tuned model
    model.save_pretrained('fine_tuned_bert_for_qa')

if __name__ == "__main__":
    main()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
Train loss: 3.038824931691893
Validation loss: 2.2950496969596257
Epoch duration: 0:33:19.483891
Epoch 2/2
Train loss: 1.9837962798805135
Validation loss: 1.9781520093581293
Epoch duration: 0:33:08.554632


In [26]:
def answer_questions_from_pdf(model, tokenizer, questions, sentences, device):
    answers = []
    for question in questions:
        answers_per_question = []
        for sentence in sentences:
            inputs = tokenizer(question, sentence, return_tensors="pt", max_length=384, truncation=True)
            inputs = inputs.to(device)
            with torch.no_grad():
                outputs = model(**inputs)

            # Get the predicted answer span
            answer_start = torch.argmax(outputs.start_logits)
            answer_end = torch.argmax(outputs.end_logits) + 1

            # Convert token indices to answer string
            answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

            answers_per_question.append(answer)
        answers.append(answers_per_question)

    return answers



In [55]:
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering
from nltk.tokenize import sent_tokenize
import fitz  # PyMuPDF for PDF extraction

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Preprocess text by removing unnecessary characters
def preprocess_text(text):
    text = text.replace("\n", " ")
    text = text.replace("\x0c", "")
    return text.strip()

# Function to answer a single question from a list of sentences
def answer_question(model, tokenizer, question, sentences, device):
    best_answer = None
    max_score = float('-inf')
    for sentence in sentences:
        # Tokenize inputs
        inputs = tokenizer(question, sentence, return_tensors="pt", max_length=384, truncation=True)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Perform inference
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Retrieve answer indices
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1

        # Convert token indices to answer text
        answer = tokenizer.decode(input_ids[0][start_idx:end_idx])


        # Calculate total score
        score = outputs.start_logits[0, start_idx] + outputs.end_logits[0, end_idx - 1]

        # Update best answer if current answer has higher score
        if score > max_score:
            best_answer = answer
            max_score = score

    return best_answer

def main():
    # Path to PDF file
    pdf_path = 'path to pdf.pdf'  # Replace with your PDF file path

    # Load fine-tuned tokenizer and model
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    model = BertForQuestionAnswering.from_pretrained('fine_tuned_bert_for_qa')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # Extract text from PDF and preprocess
    pdf_text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(pdf_text)
    sentences = sent_tokenize(cleaned_text)

    # Define questions to ask
    questions = [
        "what are solutions?",
        "Who is the author?",
        "What are the key findings?"
    ]

    # Answer each question from the PDF
    for question in questions:
        answer = answer_question(model, tokenizer, question, sentences, device)
        print(f"Question: {question}")
        print(f"Answer: {answer}\n")

if __name__ == "__main__":
    main()


Question: what are solutions?
Answer: risks associated with the solution, including technical, operational

Question: Who is the author?
Answer: tasks, the

Question: What are the key findings?
Answer: ending tasks,

